In [8]:
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
JJ_path = '/content/drive/MyDrive/IPRO497_Demographic Team/Final/data/'
K_path = '/Users/kunwoosmac/Library/Mobile Documents/com~apple~CloudDocs/Python Workspace/IIT Courses/TIF DS Project/Final/data/'
L_path = '/content/drive/MyDrive/IPRO/IPRO 497 TIF Districts/IPRO497_Demographic Team/Final/data/'

path = JJ_path

In [22]:

def process_data_and_save(owner_filepath, renter_filepath, racial_population_filepath, data_filepath):
    # Load racial population data
    df = pd.read_csv(racial_population_filepath)

    # Calculate racial ratios
    total = df['Total Population']
    white_pop = df['White Population']
    black_pop = df['Black Population']
    his_pop = df['Hispanic Population']

    white_ratios = [white / total if total != 0 else 0 for white, total in zip(white_pop, total)]
    black_ratios = [black / total if total != 0 else 0 for black, total in zip(black_pop, total)]
    his_ratios = [his / total if total != 0 else 0 for his, total in zip(his_pop, total)]

    df['White Ratio'] = white_ratios
    df['Black Ratio'] = black_ratios
    df['Hispanic Ratio'] = his_ratios

    # If the name of data file contain 'house', calculate owner and renter rate each race
    if 'house' in data_filepath:
      # Merge with housing data
      house = pd.read_csv(data_filepath)
      house = house.iloc[1:, :]
      merged_df = pd.merge(df, house, left_on='geoid', right_on='GEO_ID')

      # Calculate owner and renter populations
      merged_df['white_owner'] = merged_df['B25003_002E'].astype(float) * merged_df['White Ratio'].astype(float)
      merged_df['black_owner'] = merged_df['B25003_002E'].astype(float) * merged_df['Black Ratio'].astype(float)
      merged_df['his_owner'] = merged_df['B25003_002E'].astype(float) * merged_df['Hispanic Ratio'].astype(float)

      merged_df['white_rent'] = merged_df['B25003_003E'].astype(float) * merged_df['White Ratio'].astype(float)
      merged_df['black_rent'] = merged_df['B25003_003E'].astype(float) * merged_df['Black Ratio'].astype(float)
      merged_df['his_rent'] = merged_df['B25003_003E'].astype(float) * merged_df['Hispanic Ratio'].astype(float)

      # Create owner and renter dataframes
      owner_data = {
          'GEO_ID': merged_df['geoid'],
          'Geographic Area Name': merged_df['Geographic Area Name'],
          'Owner': merged_df[['white_owner', 'black_owner', 'his_owner']].values.tolist()
      }

      renter_data = {
          'GEO_ID': merged_df['geoid'],
          'Geographic Area Name': merged_df['Geographic Area Name'],
          'Renter': merged_df[['white_rent', 'black_rent', 'his_rent']].values.tolist()
      }

      owner_df = pd.DataFrame(owner_data)
      renter_df = pd.DataFrame(renter_data)

      # Save dataframes as pickle files
      owner_df.to_pickle(owner_filepath)
      renter_df.to_pickle(renter_filepath)

    # If the name of data file contain 'employment', calculate employment and unemployment rate each race
    elif 'employment' in data_filepath:
      print('hi')
      emp = pd.read_csv(data_filepath)
      emp = emp.iloc[1:, :]
      merged_emp = pd.merge(df, emp, left_on='geoid', right_on='GEO_ID')

      # Calculate owner and renter populations
      merged_emp['white_emp'] = merged_emp['B23025_002E'].astype(float) * merged_emp['White Ratio'].astype(float)
      merged_emp['black_emp'] = merged_emp['B23025_002E'].astype(float) * merged_emp['Black Ratio'].astype(float)
      merged_emp['his_emp'] = merged_emp['B23025_002E'].astype(float) * merged_emp['Hispanic Ratio'].astype(float)

      merged_emp['white_unemp'] = merged_emp['B23025_007E'].astype(float) * merged_emp['White Ratio'].astype(float)
      merged_emp['black_unemp'] = merged_emp['B23025_007E'].astype(float) * merged_emp['Black Ratio'].astype(float)
      merged_emp['his_unemp'] = merged_emp['B23025_007E'].astype(float) * merged_emp['Hispanic Ratio'].astype(float)

      # Create owner and renter dataframes
      emp_data = {
          'GEO_ID': merged_emp['geoid'],
          'Geographic Area Name': merged_emp['Geographic Area Name'],
          'Employment': merged_emp[['white_emp', 'black_emp', 'his_emp']].values.tolist()
      }

      unemp_data = {
          'GEO_ID': merged_emp['geoid'],
          'Geographic Area Name': merged_emp['Geographic Area Name'],
          'Unemployment': merged_emp[['white_unemp', 'black_unemp', 'his_unemp']].values.tolist()
      }

      emp_df = pd.DataFrame(emp_data)
      unemp_df = pd.DataFrame(unemp_data)

      # Save dataframes as pickle files
      emp_df.to_pickle(owner_filepath)
      unemp_df.to_pickle(renter_filepath)

    else:
      print("There is no file name contained 'house' or 'employment'")

In [23]:
for year in ['2017','2018','2019','2020','2021','2022']:
  owner_filepath = path+f'{year}_employed.pickle'
  renter_filepath = path+f'{year}_unemployed.pickle'
  racial_population_filepath = path+'racial_population.csv'
  housing_data_filepath = path+f'{year}_employment.csv'

  process_data_and_save(owner_filepath, renter_filepath, racial_population_filepath, housing_data_filepath)

  # with open(path+f'{year}_owner.pickle', 'rb') as f:
  #   owner = pickle.load(f)
  # print(owner["Owner"].iloc[0])


hi
hi
hi
hi
hi
hi


In [24]:
import pickle
with open(path+'2019_employed.pickle', 'rb') as f:
	housing_price = pickle.load(f)

In [25]:
housing_price

,GEO_ID,Geographic Area Name,Employment
0,1500000US170310101001,"Block Group 1, Census Tract 101","[125.22909090909091, 83.81090909090909, 58.96]"
1,1500000US170310101002,"Block Group 2, Census Tract 101","[188.5838896306186, 811.3609256786827, 124.055..."
2,1500000US170310101003,"Block Group 3, Census Tract 101","[657.1362162162162, 442.4351351351351, 169.428..."
3,1500000US170310102011,"Block Group 1, Census Tract 102.01","[260.31345565749234, 380.914373088685, 220.772..."
4,1500000US170310102012,"Block Group 2, Census Tract 102.01","[397.6647525293957, 691.2893081761007, 602.045..."
...,...,...,...
3988,1500000US170318439004,"Block Group 4, Census Tract 8439","[9.495253164556962, 342.94620253164555, 0.5585..."
3989,1500000US170318439005,"Block Group 5, Census Tract 8439","[17.0197904540163, 316.25145518044235, 6.72875..."
3990,1500000US170319800001,"Block Group 1, Census Tract 9800","[0.0, 0.0, 0.0]"
3991,1500000US170319801001,"Block Group 1, Census Tract 9801","[0.0, 0.0, 0.0]"


In [ ]:
housing_price

,GEO_ID,Geographic Area Name,Owner
0,1500000US170310101001,"Block Group 1, Census Tract 101","[58.409090909090914, 39.09090909090909, 27.5]"
1,1500000US170310101002,"Block Group 2, Census Tract 101","[36.911437472185135, 158.80729862038274, 24.28..."
2,1500000US170310101003,"Block Group 3, Census Tract 101","[239.75891891891894, 161.42432432432432, 61.81..."
3,1500000US170310102011,"Block Group 1, Census Tract 102.01","[143.74617737003058, 210.34250764525993, 121.9..."
4,1500000US170310102012,"Block Group 2, Census Tract 102.01","[21.635220125786166, 37.61006289308176, 32.754..."
...,...,...,...
3865,1500000US170318439004,"Block Group 4, Census Tract 8439","[3.2816455696202533, 118.5253164556962, 0.1930..."
3866,1500000US170318439005,"Block Group 5, Census Tract 8439","[3.7043073341094295, 68.83119906868451, 1.4644..."
3867,1500000US170319800001,"Block Group 1, Census Tract 9800","[0.0, 0.0, 0.0]"
3868,1500000US170319801001,"Block Group 1, Census Tract 9801","[0.0, 0.0, 0.0]"
